# Documentation

This analysis tools helps to visualize & simulate the stagnation pressure at a paraglider wing.

Inputs:
- INPUT_DIRECTORY: path to the directory containing .igc files
- OUTPUT_DIRECTORY: output data will be exported to this directory
- FILE_EXTENSION: set to `.igc`

# Imports

In [ ]:
import os
import sys
import pandas as pd
from datetime import datetime
from typing import List, Tuple

# AI content (GitHub Copilot, 02/07/2024), verified and adapted by Nicolas Huber.
src_directory: str = os.path.join(os.getcwd(), "..")
sys.path.append(src_directory)

import constants as constants
import helpers.file_processor as file_processor
import helpers.data_visualizer as data_visualizer
import algorithms.speed_analyzer as speed_analyzer
import algorithms.c_values_analyzer as c_values_analyzer
import algorithms.pressure_analyzer as pressure_analyzer

# Variables

In [ ]:
INPUT_DIRECTORY: str = "INPUT_DIRECTORY/" # end with "/
OUTPUT_DIRECTORY: str = "OUTPUT_DIRECTORY" # end with /
FILE_EXTENSION: str = ".igc"

# Initialisation

In [ ]:
file_processor = file_processor.FileProcessor()
speed_analyzer = speed_analyzer.SpeedAnalyzer()
data_visualizer = data_visualizer.DataVisualizer()
c_analyzer = c_values_analyzer.CAnalyzer()
p_analyzer = pressure_analyzer.PressureAnalyzer()

theoretical_reference: pd.DataFrame = pd.read_csv(f"{constants.THEORETICAL_REFERENCE_PATH}")

timestamp: str = datetime.now().strftime("%Y%m%d-%H%M%S")

# File Listing

In [ ]:
file_paths: List[str] = file_processor.get_file_paths(path=INPUT_DIRECTORY, file_extension=FILE_EXTENSION)

print(f"Importing files for the following conditions:")
print(f"--> Directory: {INPUT_DIRECTORY}")
print(f"--> File extension: {FILE_EXTENSION}")
print(f"--> Theoretical polar: {constants.THEORETICAL_REFERENCE_PATH.split("/")[-1]}")
print(f"--> Original reference: {constants.ORIGINAL_REFERENCE_PATH.split("/")[-1]}")
print()
print("Files:")
print(f"--> Found {len(file_paths)} files.")
print(f"--> The processing is initiated.")

# File Processing

In [ ]:
data_raw: pd.DataFrame = speed_analyzer.process_raw_data(file_paths=file_paths)

# Data Preprocessing

In [ ]:
# igc data
theoretical_reference_filtered: pd.DataFrame = speed_analyzer.filter_raw_data(data=theoretical_reference, reference=True)
data_raw_filtered: pd.DataFrame = speed_analyzer.filter_raw_data(data=data_raw)

print(f"Filtered data:")
print(f"--> Data points for filtered theoretical reference: {len(theoretical_reference_filtered)} (lost {len(theoretical_reference) - len(theoretical_reference_filtered)})")
print(f"--> Data points for filtered tracklogs: {len(data_raw_filtered)} (lost {len(data_raw) - len(data_raw_filtered)})")
print()

# smooth and filter igc data
smoothed_data_raw: pd.DataFrame = speed_analyzer.savgol_filter(data=data_raw_filtered)
smoothed_data_grouped: pd.DataFrame = speed_analyzer.group_data(data=smoothed_data_raw)

print(f"Data smoothing:")
print(f"--> During smoothing, the raw data points were reduced from {len(data_raw_filtered)} to {len(smoothed_data_raw)} (lost {len(data_raw_filtered) - len(smoothed_data_raw)}).")
print(f"--> During grouping, the smoothed data points were reduced from {len(smoothed_data_raw)} to {len(smoothed_data_grouped)} (lost {len(smoothed_data_raw) - len(smoothed_data_grouped)}).")
print()

# add airspeed to datasets
print(f"Manipulating datasets:")
print(f"--> The datasets are being extended:")
print(f"----> The theoretical reference dataset is being extended with the airspeed, that's calculated based on horizontal and vertical speed.")
print(f"----> The experimental dataset is being extended with the airspeed, that's calculated based on horizontal and vertical speed.")
print(f"--> Vertical speeds are converted to be positive:")
print(f"----> The vertical speeds of the theoretical reference dataset are being converted to be positive.")
print(f"----> The vertical speeds of the experimental dataset are being converted to be positive.")

airspeed_theoretical_reference: pd.DataFrame = c_analyzer.positive_vertical_speed(speed_data=c_analyzer.calculate_airspeed(speed_data=theoretical_reference_filtered))
airspeed_smoothed_data_grouped: pd.DataFrame = c_analyzer.positive_vertical_speed(speed_data=c_analyzer.calculate_airspeed(speed_data=smoothed_data_grouped))

print(f"--> The datasets have been extended and manipulated.")
print()

# calculate c values for datasets (simplified and optimized algorithm)
print(f"C values:")
print(f"--> Calculating the c values for the following conditions:")
print(f"----> Altitude: {constants.ALTITUDE} m")
print(f"----> Air density: {constants.AIR_DENSITY} kg/m^3")
print(f"----> Gravity: {constants.GRAVITY} m/s^2")
print(f"----> Wing area: {constants.WING_AREA} m^2")
print(f"----> Takeoff mass: {constants.MASS} kg")
print(f"--> Calculating the c values for the following datasets:")
print(f"----> The c values are being calculated for the original reference dataset (simplified & optimized algorithm).")
print(f"----> The c values are being calculated for the theoretical reference dataset (simplified & optimized algorithm).")
print(f"----> The c values are being calculated for the experimental dataset (simplified & optimized algorithm).")

c_values_theoretical_reference_optimized: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_theoretical_reference, algorithm=True)
c_values_experimental_optimized: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_smoothed_data_grouped, algorithm=True)

print(f"--> The c values have been calculated.")

# Pressure Analysis

In [ ]:
print(f"Pressure analysis:")
print(f"--> The pressure data is being processed for the theoretical reference dataset.")
print(f"--> The pressure data is being processed for the experimental dataset.")
print()

pressure_data_theoretical: pd.DataFrame = p_analyzer.process_pressure_data(data=c_values_theoretical_reference_optimized)
pressure_data_experimental: pd.DataFrame = p_analyzer.process_pressure_data(data=c_values_experimental_optimized)

print(f"Export to CSV:")
print(f"The theoretical reference dataset is being exported to a CSV file: {OUTPUT_DIRECTORY}{timestamp}_SJf_dynamic-pressure_theoretical-reference_nicolas-huber.csv")
print(f"The experimental dataset is being exported to a CSV file: {OUTPUT_DIRECTORY}{timestamp}_SJf_dynamic-pressure_experimental_nicolas-huber.csv")

p_analyzer.export_to_csv(data=pressure_data_theoretical, path=f"{OUTPUT_DIRECTORY}{timestamp}_SJf_dynamic-pressure_theoretical-reference_nicolas-huber.csv")
p_analyzer.export_to_csv(data=pressure_data_experimental, path=f"{OUTPUT_DIRECTORY}{timestamp}_SJf_dynamic-pressure_experimental_nicolas-huber.csv")

# Pressure Visualisation

In [ ]:
data_visualizer.visualize_pressure(experimental_data=pressure_data_experimental, theoretical_data=pressure_data_theoretical, title="Experimentelle Modellierung des Staudrucks am Flügel")
data_visualizer.visualize_pressure_deviation(experimental_data=pressure_data_experimental, theoretical_data=pressure_data_theoretical, title="Abweichung des Staudrucks am Flügel")

# System Info

In [ ]:
print(f"@ Author {constants.AUTHOR}")
print(f"@ Author Email {constants.AUTHOR_EMAIL}")
print(f"@ Author URL {constants.AUTHOR_URL}")
print(f"@ GitHub URL {constants.GITHUB_URL}")